# 1D ROIs
We have already adjusted for the number of frames (see n_frames_adjustment_1D). Now we need to adjust for the image size and the amplitude of the signal. 
## Adjusting for image size
The issue is that if we had the same image content (same cells) but the image size was different, the spatial distance would be higher and thus weighted more heavily. We want to adjust for this. The way we do this is by taking a pixels per centimeter (dots per inch) value. We then calculate the centers based on this, not the pixel values.

In [1]:
# init
import numpy as np
import pandas as pd
from statistics import mean

# vars
n_frames = 5

class Roi1D:
    N_HORIZONTAL: int = None
    PIXELS_PER_CM: int = None
    WIDTH_PX: int = None  # The width in pixels
    

    @classmethod
    @property
    def width_cm(cls) -> float:
        """The width in cm"""
        return cls.WIDTH_PX / cls.PIXELS_PER_CM  

    def __init__(self, x_idx):
        self.x_idx = x_idx

    def __repr__(self):
        return f'Roi1D({self.x_idx})'

    def __str__(self):
        return f'Roi1D({self.x_idx})'

    def coordinates_pixels(self):
        """Inclusive lower and upper pixel indexes"""
        left = self.x_idx * Roi1D.WIDTH_PX
        return [left, left + Roi1D.WIDTH_PX - 1]

    def center_pixels(self):
        return mean(self.coordinates_pixels())

    # -------------------- This is the important part: --------------------------------
    def coordinates_cm(self):
        """The left and right coordinates in cm"""
        left = self.x_idx * Roi1D.width_cm
        right = left + Roi1D.width_cm
        return [left, right]

    def center_cm(self):
        """Center in cm"""
        return mean(self.coordinates_cm())
    # ------------------------------------------------------------------------------

In [2]:
Roi1D.PIXELS_PER_CM = 5
Roi1D.WIDTH_PX = 20
Roi1D.width_cm

4.0

In [3]:
def make_vars(image_width, n_horizontal, center_in_cm: bool):
    Roi1D.N_HORIZONTAL = n_horizontal
    Roi1D.WIDTH_PX = image_width // Roi1D.N_HORIZONTAL
    rois = [Roi1D(x) for x in range(Roi1D.N_HORIZONTAL)]
    signals_df = pd.DataFrame(
        {roi: np.full(shape=n_frames, fill_value=1) for roi in rois}
    )
    if center_in_cm:
        spatial_indexes = pd.DataFrame(
            {roi: [roi.center_cm() * np.sqrt(n_frames)] for roi in rois},
            index=['center'])
    else:
        spatial_indexes = pd.DataFrame(
            {roi: [roi.center_pixels() * np.sqrt(n_frames)] for roi in rois},
            index=['center'])
    signals_indexes = pd.concat([spatial_indexes, signals_df])
    return rois, signals_df, spatial_indexes, signals_indexes

In [4]:
from scipy.spatial.distance import euclidean


def add_distances_to_df(dists, image_width, roi0, roi1, signals_df, spatial_indexes, signals_indexes):
    dists.loc[image_width, 'ROI Width'] = roi0.WIDTH_PIXELS
    dists.loc[image_width, 'Original Signal Distance'] = euclidean(signals_df[roi0], signals_df[roi1])
    dists.loc[image_width, 'Spatial Distance'] = euclidean(spatial_indexes[roi0], spatial_indexes[roi1])
    dists.loc[image_width, 'Signals and spatial indexes Distance'] = euclidean(signals_indexes[roi0],
                                                                               signals_indexes[roi1])

### Without image size adjustment

In [5]:
_, _, _, signals_indexes = make_vars(image_width=20, n_horizontal=4, center_in_cm=False)
signals_indexes

,Roi1D(0),Roi1D(1),Roi1D(2),Roi1D(3)
center,4.472136,15.652476,26.832816,38.013156
0,1.000000,1.000000,1.000000,1.000000
1,1.000000,1.000000,1.000000,1.000000
2,1.000000,1.000000,1.000000,1.000000
3,1.000000,1.000000,1.000000,1.000000
4,1.000000,1.000000,1.000000,1.000000


In [6]:
# When we keep the number of ROIs constant but increase the image size, the distance between the ROIs increases, even though the ROI grid is the equivalent.
n_horizontal = 4

distances = pd.DataFrame()
for image_width in [8, 20, 100, 200, 1_000, 10_000]:
    rois, signals_df, spatial_indexes, signals_indexes = make_vars(image_width=image_width, n_horizontal=n_horizontal,
                                                                   center_in_cm=False)
    add_distances_to_df(distances, image_width, rois[0], rois[1], signals_df, spatial_indexes, signals_indexes)
distances

,ROI Width,Original Signal Distance,Spatial Distance,Signals and spatial indexes Distance
8,2.0,0.0,4.472136,4.472136
20,5.0,0.0,11.180340,11.180340
100,25.0,0.0,55.901699,55.901699
200,50.0,0.0,111.803399,111.803399
1000,250.0,0.0,559.016994,559.016994
10000,2500.0,0.0,5590.169944,5590.169944


### With image size adjustment

In [7]:
Roi1D.PIXELS_PER_CM = 5
_, _, _, signals_indexes = make_vars(20, 4, center_in_cm=True)
signals_indexes

,Roi1D(0),Roi1D(1),Roi1D(2),Roi1D(3)
center,1.118034,3.354102,5.59017,7.826238
0,1.000000,1.000000,1.00000,1.000000
1,1.000000,1.000000,1.00000,1.000000
2,1.000000,1.000000,1.00000,1.000000
3,1.000000,1.000000,1.00000,1.000000
4,1.000000,1.000000,1.00000,1.000000


In [8]:
n_horizontal = 4
image_width_cm = 10
image_width_px_arr = np.array([8, 20, 100, 200, 1_000, 10_000])
rois_ = [Roi1D(x) for x in range(n_horizontal)]
centers = pd.DataFrame()
distances = pd.DataFrame()

for i, image_width in enumerate(image_width_px_arr):
    Roi1D.PIXELS_PER_CM = image_width / image_width_cm
    rois, signals_df, spatial_indexes, signals_indexes = make_vars(image_width=image_width, n_horizontal=n_horizontal,
                                                                   center_in_cm=True)
    add_distances_to_df(distances, image_width, rois[0], rois[2], signals_df, spatial_indexes, signals_indexes)
    
    spatial_indexes.index = [image_width]
    spatial_indexes.columns = rois_
    centers = pd.concat([centers, spatial_indexes])
distances

,ROI Width,Original Signal Distance,Spatial Distance,Signals and spatial indexes Distance
8,2.0,0.0,11.18034,11.18034
20,5.0,0.0,11.18034,11.18034
100,25.0,0.0,11.18034,11.18034
200,50.0,0.0,11.18034,11.18034
1000,250.0,0.0,11.18034,11.18034
10000,2500.0,0.0,11.18034,11.18034


In [9]:
centers

,Roi1D(0),Roi1D(1),Roi1D(2),Roi1D(3)
8,2.795085,8.385255,13.975425,19.565595
20,2.795085,8.385255,13.975425,19.565595
100,2.795085,8.385255,13.975425,19.565595
200,2.795085,8.385255,13.975425,19.565595
1000,2.795085,8.385255,13.975425,19.565595
10000,2.795085,8.385255,13.975425,19.565595


In [19]:
image_width_cm = 8
Roi1D.N_HORIZONTAL = 2
rois = [Roi1D(x) for x in range(Roi1D.N_HORIZONTAL)]
image_width_px_arr = np.array([4, 8, 12, 16, 20, 40, 80, 120, 200, 400])

results = pd.DataFrame(columns=['Roi width (pixels)','Roi width (cm)','Pixels per cm', *rois], index=image_width_px_arr)

for image_width in image_width_px_arr:
    Roi1D.WIDTH_PX = image_width / Roi1D.N_HORIZONTAL
    Roi1D.PIXELS_PER_CM = image_width / image_width_cm
    
    results.loc[image_width, 'Roi width (pixels)'] = Roi1D.WIDTH_PX
    results.loc[image_width, 'Roi width (cm)'] = Roi1D.width_cm
    results.loc[image_width, 'Pixels per cm'] = Roi1D.PIXELS_PER_CM
    for roi in rois:
        results.loc[image_width, roi] = roi.center_cm()
        
results

,Roi width (pixels),Roi width (cm),Pixels per cm,Roi1D(0),Roi1D(1)
4,2.0,4.0,0.5,2.0,6.0
8,4.0,4.0,1.0,2.0,6.0
12,6.0,4.0,1.5,2.0,6.0
16,8.0,4.0,2.0,2.0,6.0
20,10.0,4.0,2.5,2.0,6.0
40,20.0,4.0,5.0,2.0,6.0
80,40.0,4.0,10.0,2.0,6.0
120,60.0,4.0,15.0,2.0,6.0
200,100.0,4.0,25.0,2.0,6.0
400,200.0,4.0,50.0,2.0,6.0
